# MD2DS - Master in Data Science and Statistical Learning

**Numerical Calculus and Linear Algebra**

Exercise: SOR

Deadline: 31/3/2022

In [1]:
import numpy as np
from scipy.sparse import csr_matrix, diags, tril
from scipy.sparse.linalg import spsolve_triangular

# Exercise

Consider a linear system $A\mathbf{x} = \mathbf{b}$.
1.  Write a python function `my_SOR(A, x0, b, omega, tol, kmax)` that takes in input

  -  $A\in\mathbb{R}^{m\times m}$  : square matrix
  -  $\mathbf{x}_0\in\mathbb{R}^m$   : a first approximation of the system solution
  -  $\mathbf{b}\in\mathbb{R}^m$   : costant vector
  -  $\omega\in(0,2)$                      : the relxation parameter
  -  $\texttt{tol}\in\mathbb{R}$   : a given tolerance
  -  $\texttt{kmax}\in\mathbb{N}$  : a maximum number of iterations

  and returns
  -  $\mathbf{x}\in\mathbb{R}^m$ : the approximation of the solution of the linear system, obtained by implementing the Successive Over Relaxation method;
  -  $\texttt{it}$ : the number of the computed iterations.

2.  Let
\begin{equation}
      T_n = \pmatrix{ 3       & -1     &        & \huge 0 \\
                     -1       & 3      & \ddots & \\
                              & \ddots & \ddots & -1 \\
                     \huge 0 &      & -1      & 3  }\in\mathbb{R}^{n \times n}
\end{equation}

  and define the sparse (possibly use the sparse format: $\texttt{csr_matrix}$) matrix $A\in\mathbb{R}^{n^2\times n^2}$ as
      \begin{equation}
        A = T_n \otimes I_n + I_n \otimes T_n.
      \end{equation}

  For $n=10,20,\dots,50$ solve the linear system $A\mathbf{x} = \mathbf{b}$ with exact solution $\mathbf{x} = \pmatrix{1\\2\\\vdots\\n^2}\in\mathbb{R}^{n^2}$ using the funcion `my_SOR` implemented at point (1) with
  - $\mathbf{x}_0 = \pmatrix{0\\\vdots\\0}\in\mathbb{R}^{n^2}$
  - $\omega = 0.8, 1, 1.2$
  - $\texttt{toll} = 10^{-5}$
  - $\texttt{kmax} = 10^{5}$
  
  Print
  -  the condition number of $A$,
  -  the number of computed iterations $\texttt{it}$,
  -  the relative error. 

In [2]:
def my_SOR(A, x0, b, om, toll, kmax):
    
    n    = len(b)
    
    nrb  = np.linalg.norm(b)
    
    go   = True
    
    it   = 1
    
    A    = A.reshape(n,n).toarray()
    
    Msor =  diags(np.diag(A)) + (om * tril(A, k = -1, format = "csr"))
    
    while it <= kmax and go:
        
        r  = A.dot(x0) - b      
        go = (np.linalg.norm(r) > toll * nrb)
        
        if go:
            
            it += 1   
            x0 = x0 + spsolve_triangular(Msor, om * -r, lower = True)  # lower = True : use only data contained in the lower triangle of A
    
    it -= 1
    return x0, it


In [ ]:
toll  = 1e-5
kmax  = 1e5
omega = [0.8, 1.0, 1.2]

print('-------------------------------------------------')
print('n         cond        omega     it       err_r ')
print('-------------------------------------------------')

for n in range(10, 60, 10):  #(start, stop, step)
        
    T     = np.diag(3 * np.ones(n)) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
    A     = np.kron(T, np.eye(n)) + np.kron(np.eye(n), T)        
    c     = np.linalg.cond(A)
    A     = csr_matrix(A)
    x0    = np.zeros(n ** 2)
    xex   = np.arange(1, n ** 2 + 1)  # exact solution
    b     = A.dot(xex)
    
    for om in omega:
        x, it = my_SOR(A, x0, b, om, toll, kmax)

        print(f'{n:2}        {c:.2e}    {om}     {it:4}       {np.linalg.norm(x - xex) / np.linalg.norm(xex):.2e}')

print('-------------------------------------------------')


-------------------------------------------------
n         cond        omega     it       err_r 
-------------------------------------------------
10        4.55e+00    0.8       23       1.09e-05
10        4.55e+00    1.0       15       1.01e-05
10        4.55e+00    1.2       12       1.57e-06
20        4.87e+00    0.8       25       8.08e-06
20        4.87e+00    1.0       16       1.01e-05
20        4.87e+00    1.2       11       4.13e-06
30        4.94e+00    0.8       25       9.48e-06
30        4.94e+00    1.0       17       5.85e-06
30        4.94e+00    1.2       11       4.55e-06
40        4.96e+00    0.8       25       1.02e-05
40        4.96e+00    1.0       17       6.29e-06
40        4.96e+00    1.2       11       4.80e-06
